# Election models
> Tools for election modeling, including plots spcifically designed for that

In [ ]:
#| default_exp election_models

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| exporti
import json, os, inspect
import itertools as it
from collections import defaultdict

import numpy as np
import pandas as pd
import datetime as dt

from typing import List, Tuple, Dict, Union, Optional

import altair as alt
import scipy.stats as sps

from salk_toolkit.utils import *
from salk_toolkit.io import extract_column_meta, read_json
from salk_toolkit.plots import stk_plot

# Machinery

In [ ]:
#| export

# Vectorized basic election simulation: quotas, dHondt
# Input 'support' should be of shape (draws,districts,parties)
def simulate_election(support, nmandates, threshold=0.05, quotas=True, first_quota_coef=1.0, dh_power=1.0, body_size=None, max_comp=None, **kwargs):

    # Remove parties below a threshold in uzsim_t
    zero_mask = (support.sum(axis=1)/(support.sum(axis=(1,2))+1e-3)[:,None])>threshold
    uzsim_t = zero_mask[:,None,:]*support

    # Calculate district mandates
    if quotas:
        quotas = (support.sum(axis=-1)+1e-3)/nmandates[None,:]
        v, r = np.divmod(uzsim_t/quotas[:,:,None],1.0)
        dmandates = v+(r>=first_quota_coef)
    else: dmandates = np.zeros_like(uzsim_t)

    # Calculate votes and mandates for each party
    pvotes = uzsim_t.sum(axis=1)
    pmand = dmandates.sum(axis=1)

    # Calculate d'Hondt values and get party indices out
    if max_comp is None: max_comp = body_size # This can be set lower for more efficient computation
    dhvals = pvotes[:,:,None]/(pmand[:,:,None]+np.arange(1, max_comp+1, 1)[None,None,:])**dh_power
    sinds = np.argsort(-dhvals.reshape( (dhvals.shape[0],-1) ),axis=1) // max_comp

    # Select the first n as compensation
    if body_size is None: body_size = sum(nmandates)
    rmand = body_size - pmand.sum(axis=1)
    ri = ((np.arange(sinds.shape[1])[None,:]-rmand[:,None])<0)
    comp_ident = np.concatenate([np.zeros( (1,support.shape[-1]) ),np.identity(support.shape[-1])])
    comp_mandates = comp_ident[(sinds+1)*ri].sum(axis=1)

    # Combine districts + compensation
    return np.concatenate( [dmandates,comp_mandates[:,None,:]],axis=1 )

In [ ]:
#| export

# Basic wrapper around simulate elections that goes from dataframe to dataframe
def simulate_election_e2e(sdf, parties, mandates_dict, **kwargs):
    
    # Convert data frame to a numpy tensor for fast vectorized processing
    parties = [ p for p in parties if p in sdf.columns ]
    ed_df = sdf.groupby(['draw','electoral_district'])[parties].sum()
    districts = list(sdf.electoral_district.unique())
    support = ed_df.reset_index(drop=True).to_numpy().reshape( (-1,len(districts),len(parties)) )    
    nmandates = np.array([ mandates_dict[d] for d in districts ])
    
    edt = simulate_election(support, nmandates, **kwargs)
    
    # Shape it back into a data frame
    eddf = pd.DataFrame( edt.reshape( (-1,) ), columns=['mandates'], dtype='int')
    eddf.loc[:, ['draw','electoral_district', 'party']] = np.array(tuple(it.product( range(edt.shape[0]), districts+['Compensation'], parties )))
    return eddf

In [ ]:
# Test simulation
from salk_toolkit.io import read_annotated_data, extract_column_meta

df, meta = read_annotated_data('../data/mrp.parquet')
cmeta = extract_column_meta(meta)
edm = cmeta['electoral_district']

#electoral_system = edm['electoral_system']
electoral_system = {
    "body_size": 101,
    "threshold": 0.05,
    "first_quota_coef": 0.75,
    "dh_power": 0.9,
    "max_comp": 20
}

sdf = simulate_election_e2e(df,cmeta['party_preference']['categories'],edm['mandates'],**electoral_system)
assert (sdf.groupby('draw')['mandates'].sum() == electoral_system['body_size']).all()

In [ ]:
#| export

# This fits into the pp framework a bit awkwardly: question=party, factor=electoral_district, hence the as_is and hidden flags
@stk_plot('mandate_plot', data_format='longform', continuous=True, question=True, draws=True, requires_factor=True, agg_fn='sum', factor_meta=['mandates','electoral_system'], as_is=True, hidden=True)
def mandate_plot(data, electoral_system, mandates, value_col='value', question_color_scale=alt.Undefined, question_order=alt.Undefined, factor_col=None, factor_order=alt.Undefined, width=None, alt_properties={}, outer_factors=[]):
    
    if outer_factors: raise Exception("This plot does not work with extra factors")
    
    # Reshape input to (draws,electoral_districts,parties)
    draws = data.draw.unique()
    pdf = data.pivot(index=['draw',factor_col], columns='question', values=value_col).reset_index()
    ded = pd.DataFrame(list(it.product(draws,factor_order)),columns=['draw',factor_col])
    sdata = ded.merge(pdf,on=['draw',factor_col]).loc[:,question_order].to_numpy().reshape( (len(draws),len(factor_order),len(question_order)) )
    
    # Run the actual electoral simulation
    nmandates = np.array([ mandates[d] for d in factor_order ])
    edt = simulate_election(sdata,nmandates,**electoral_system)
    
    factor_order = factor_order+['Compensation']
    
    # Shape it back into a data frame
    df = pd.DataFrame( edt.reshape( (-1,) ), columns=[value_col], dtype='int')
    df.loc[:, ['draw',factor_col, 'question']] = np.array(tuple(it.product( draws, factor_order, question_order )))
    
    maxv = df[value_col].max()
    tv = np.arange(1,maxv+1,dtype='int')[None,:]
    dfv = df[value_col].to_numpy()[:,None]
    dfm = pd.DataFrame((dfv>=tv).astype('int'),columns=tv[0], index=df.index)
    dfm['draw'],dfm['question'], dfm[factor_col] = df['draw'], df['question'], df[factor_col]
    res = dfm.groupby(['question',factor_col])[tv[0]].mean().reset_index().melt(id_vars=['question',factor_col],
                                                                                var_name='mandates',value_name=value_col)
    
    # Remove parties who have no chance of even one elector
    eliminate = (res.groupby('question')[value_col].sum() < 0.2)
    el_cols = [i for i,v in eliminate.items() if v]
    res = res[~res['question'].isin(el_cols)]
    question_order = list(eliminate[~eliminate].index)
    
    f_width = max(50,width/len(question_order))

    plot = alt.Chart(data=res).mark_bar().encode(
        x=alt.X('mandates',title=None),
        y=alt.Y(value_col,title=None,axis=alt.Axis(format='%')),
        color=alt.Color('question:N', scale=question_color_scale, legend=None),
        tooltip=[
            alt.Tooltip('question', title='party'),
            alt.Tooltip(factor_col),
            alt.Tooltip('mandates'),
            alt.Tooltip(value_col, format='.1%', title='probability'),
            ]
    ).properties(
        width=f_width,
        height=f_width//2,
        **alt_properties
        #title="Ringkonna- ja kompensatsioonimandaatide tõenäolised jaotused"
    ).facet(
        #header=alt.Header(labelAngle=-90),
        row=alt.X(
            f'{factor_col}:N',
            sort=factor_order+['Compensation'],
            title=None,
            header=alt.Header(labelOrient='top')
            ),
        column=alt.Y(
            'question:N',
            sort=question_order,
            title=None,
            header=alt.Header(labelFontWeight='bold')
            ),
    )
    return plot

In [ ]:
# Test the plot

from salk_toolkit.pp import e2e_plot
e2e_plot({
    'res_col' : 'party_preference_dist',
    'factor_cols': ['electoral_district'],
    'internal_facet': True,
    'plot': 'mandate_plot',
    #'agg_fn': 'sum'
}, '../data/mrp.parquet', width=800)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()